# 1) Data Import and Initial Inspection

In [6]:
import pandas as pd
df = pd.read_csv('Short-Data.csv')
df

,tweet,Label
0,`- This is not ``creative``. Those are the di...,0
1,` :: the term ``standard model`` is itself le...,0
2,"True or false, the situation as of March 200...",0
3,"Next, maybe you could work on being less cond...",0
4,This page will need disambiguation.,0
...,...,...
415,great president my ass that nigger faggot used...,1
416,trying to help people when i'm literally a dum...,1
417,@user ching chong sheng shiong,1
418,And now yu white nigger,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420 entries, 0 to 419
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweet   420 non-null    object
 1   Label   420 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 6.7+ KB


In [6]:
df['Label'].value_counts()

Label
0    215
1    205
Name: count, dtype: int64

In [7]:
df.duplicated().sum()

0

### 1.1) Removing Duplicates

In [8]:
df.drop_duplicates(inplace=True)

In [9]:
df.duplicated().sum()

0

In [10]:
df['Label'].value_counts()

Label
0    215
1    205
Name: count, dtype: int64

# 2) Cleaning and Preprocessing

### 2.1) Converting Tweets to Lowercase

In [11]:
df['tweet'] = df['tweet'].str.lower()
print(df)

                                                 tweet  Label
0    `- this is not ``creative``.  those are the di...      0
1    `  :: the term ``standard model`` is itself le...      0
2      true or false, the situation as of march 200...      0
3     next, maybe you could work on being less cond...      0
4                 this page will need disambiguation.       0
..                                                 ...    ...
415  great president my ass that nigger faggot used...      1
416  trying to help people when i'm literally a dum...      1
417                     @user ching chong sheng shiong      1
418                            and now yu white nigger      1
419  glad that the despicable trump fan magabomber ...      1

[420 rows x 2 columns]


### 2.2) Cleaning HTML Content

In [12]:
from bs4 import BeautifulSoup
for index, row in df.iterrows():
    cleaned_tweet = BeautifulSoup(row['tweet']).text
    df.at[index, 'tweet'] = cleaned_tweet
print(df)

                                                 tweet  Label
0    `- this is not ``creative``.  those are the di...      0
1    `  :: the term ``standard model`` is itself le...      0
2      true or false, the situation as of march 200...      0
3     next, maybe you could work on being less cond...      0
4                 this page will need disambiguation.       0
..                                                 ...    ...
415  great president my ass that nigger faggot used...      1
416  trying to help people when i'm literally a dum...      1
417                     @user ching chong sheng shiong      1
418                            and now yu white nigger      1
419  glad that the despicable trump fan magabomber ...      1

[420 rows x 2 columns]


C:\Users\salman\AppData\Local\Temp\ipykernel_11124\3149476355.py:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  cleaned_tweet = BeautifulSoup(row['tweet']).text


### 2.3) Removing URLs

In [13]:
import re
def remove_url(tweet):
    pattern = re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'', tweet)
df['tweet'] = df['tweet'].apply(remove_url)
print(df)

                                                 tweet  Label
0    `- this is not ``creative``.  those are the di...      0
1    `  :: the term ``standard model`` is itself le...      0
2      true or false, the situation as of march 200...      0
3     next, maybe you could work on being less cond...      0
4                 this page will need disambiguation.       0
..                                                 ...    ...
415  great president my ass that nigger faggot used...      1
416  trying to help people when i'm literally a dum...      1
417                     @user ching chong sheng shiong      1
418                            and now yu white nigger      1
419  glad that the despicable trump fan magabomber ...      1

[420 rows x 2 columns]


### 2.4) Removing Punctuation

In [14]:
import string
exclude = set(string.punctuation)
def remove_punctuation(tweet):
    text = ''
    for char in tweet:
        if char not in exclude:
            text += char
    return text
df['tweet'] = df['tweet'].apply(remove_punctuation)
print(df)

                                                 tweet  Label
0     this is not creative  those are the dictionar...      0
1       the term standard model is itself less npov...      0
2      true or false the situation as of march 2002...      0
3     next maybe you could work on being less conde...      0
4                  this page will need disambiguation       0
..                                                 ...    ...
415  great president my ass that nigger faggot used...      1
416  trying to help people when im literally a dumb...      1
417                      user ching chong sheng shiong      1
418                            and now yu white nigger      1
419  glad that the despicable trump fan magabomber ...      1

[420 rows x 2 columns]


### 2.5) Removing Special Characters

In [15]:
import re
def remove_special_characters(tweet):
    special_chars_pattern = re.compile(r'[^\x00-\x7F]')
    cleaned_tweet = special_chars_pattern.sub('', tweet)
    return cleaned_tweet
df['tweet'] = df['tweet'].apply(remove_special_characters)
print(df)

                                                 tweet  Label
0     this is not creative  those are the dictionar...      0
1       the term standard model is itself less npov...      0
2      true or false the situation as of march 2002...      0
3     next maybe you could work on being less conde...      0
4                  this page will need disambiguation       0
..                                                 ...    ...
415  great president my ass that nigger faggot used...      1
416  trying to help people when im literally a dumb...      1
417                      user ching chong sheng shiong      1
418                            and now yu white nigger      1
419  glad that the despicable trump fan magabomber ...      1

[420 rows x 2 columns]


### 2.6) Expanding Chat Abbreviations

In [16]:
chat_word = {
    "AFAIK": "As Far As I Know",
    "AFK": "Away From Keyboard",
    "ASAP": "As Soon as Possible",
    "ATK": "At the Keyboard",
    "ATM": "At the Moment",
    "A3": "Anytime, Anywhere, Anyplace",
    "BAK": "Back at Keyboard",
    "BBL": "Be Back Later",
    "BBS": "Be Back Soon",
    "BFN": "Bye For Now",
    "B4N": "Bye For Now",
    "BRB": "Be Right Back",
    "BRT": "Be Right There",
    "BTW": "By the Way",
    "B4": "Before",
    "CU": "See You",
    "CUL8R": "See You Later",
    "CYA": "See You",
    "FAQ": "Frequently Asked Questions",
    "FC": "Fingers Crossed",
    "FWIW": "For What It's Worth",
    "FYI": "For Your Information",
    "GAL": "Get a Life",
    "GG": "Good Game",
    "Gli": "Good Night",
    "GNTA": "Great Hinds Think Alike",
    "GB": "Great!",
    "G9": "Genius",
    "IC": "I See",
    "ICQ": "I Seek You (also a chat program)",
    "ILU": "I Love You",
    "IMHO": "In My Humble Opinion",
    "IMO": "In My Opinion",
    "10hi": "In Other Words",
    "IRL": "In Real Life",
    "KISS": "Keep It Simple, Stupid",
    "LDR": "Long Distance Relationship",
    "LMAO": "Laugh My A.. Off",
    "LOL": "Laughing Out Loud",
    "LTNS": "Long Time No See",
    "18R": "Later",
    "MTE": "My Thoughts Exactly",
    "M8": "Mate",
    "IRN": "No Reply Necessary",
    "OIC": "Oh I See",
    "PITA": "Pain In The A..",
    "PRT": "Party",
    "PRW": "Parents Are Watching",
    "QPSA?": "Que Pasa?",
    "ROFL": "Rolling On The Floor Laughing",
    "ROFLOL": "Rolling On The Floor Laughing Out Loud",
    "ROTFLMAO": "Rolling On The Floor Laughing My A.. Off",
    "SK8": "Skate",
    "STATS": "Your sex and age",
    "ASL": "Age, Sex, Location",
    "THX": "Thank You",
    "TTFH": "Ta-Ta For Now!",
    "TTVL": "Talk To You Later",
    "U": "You",
    "U * 2": "1 Too",
    "U4E": "Yours For Ever",
    "WB": "Welcome Back",
    "WTF": "What The F...",
    "WTG": "Way To Go!"
}

def chat_conversation(tweet):
    new_text = []
    for i in tweet.split():
        if i.upper() in chat_word:
            new_text.append(chat_word[i.upper()])
        else:
            new_text.append(i)
    return " ".join(new_text)
df['tweet'] = df['tweet'].apply(chat_conversation)
print(df)


                                                 tweet  Label
0    this is not creative those are the dictionary ...      0
1    the term standard model is itself less npov th...      0
2    true or false the situation as of march 2002 w...      0
3    next maybe you could work on being less condes...      0
4                   this page will need disambiguation      0
..                                                 ...    ...
415  great president my ass that nigger faggot used...      1
416  trying to help people when im literally a dumb...      1
417                      user ching chong sheng shiong      1
418                            and now yu white nigger      1
419  glad that the despicable trump fan magabomber ...      1

[420 rows x 2 columns]


### 2.7) Spelling Correction

In [ ]:
from textblob import TextBlob
def correct_spelling(tweet):
    correct_tweet = TextBlob(tweet)
    return correct_tweet.correct().string
df['tweet'] = df['tweet'].apply(correct_spelling)
print(df)

### 2.8) Removing Stop Words

In [18]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
def remove_stop_word(tweet):
    new_word = []
    for word in tweet.split():
        if word.lower() not in stopwords.words('english'):
            new_word.append(word)
    return " ".join(new_word)

df['tweet'] = df['tweet'].apply(remove_stop_word)
print(df)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\salman\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                                                 tweet  Label
0    creative dictionary definitions terms insuranc...      0
1    term standard model less npov think wed prefer...      0
2    true false situation march 2002 saudi proposal...      0
3    next maybe could work less condescending sugge...      0
4                             page need disambiguation      0
..                                                 ...    ...
415  great president ass nigger faggot used white h...      1
416  trying help people im literally dumbass cunt c...      1
417                      user ching chong sheng shiong      1
418                                    yu white nigger      1
419  glad despicable trump fan magabomber quickly f...      1

[420 rows x 2 columns]


### 2.9) Removing Emojis

In [19]:
import re
def remove_emojis(tweet):
    emoji_pattern = re.compile("["
                            u"\U0001F600-\U0001F64F"
                            u"\U0001F300-\U0001F5FF"
                            u"\U0001F680-\U0001F6FF"
                            u"\U0001F1E0-\U0001F1FF"
                            u"\U00002702-\U000027B0"
                            u"\U000024C2-\U0001F251"

    "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', tweet)

df['tweet'] = df['tweet'].apply(remove_emojis)
print(df)

                                                 tweet  Label
0    creative dictionary definitions terms insuranc...      0
1    term standard model less npov think wed prefer...      0
2    true false situation march 2002 saudi proposal...      0
3    next maybe could work less condescending sugge...      0
4                             page need disambiguation      0
..                                                 ...    ...
415  great president ass nigger faggot used white h...      1
416  trying help people im literally dumbass cunt c...      1
417                      user ching chong sheng shiong      1
418                                    yu white nigger      1
419  glad despicable trump fan magabomber quickly f...      1

[420 rows x 2 columns]


### 2.10) Tokenization

In [7]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

def tokenize_tweet(tweet):
    tokens = word_tokenize(tweet)
    return tokens
df['tokens'] = df['tweet'].apply(tokenize_tweet)
print(df)

[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


                                                 tweet  Label  \
0    `- This is not ``creative``.  Those are the di...      0   
1    `  :: the term ``standard model`` is itself le...      0   
2      True or false, the situation as of March 200...      0   
3     Next, maybe you could work on being less cond...      0   
4                 This page will need disambiguation.       0   
..                                                 ...    ...   
415  great president my ass that nigger faggot used...      1   
416  trying to help people when i'm literally a dum...      1   
417                     @user ching chong sheng shiong      1   
418                            And now yu white nigger      1   
419  Glad that the despicable Trump fan MAGABOMBER ...      1   

                                                tokens  
0    [`, -, This, is, not, ``, creative, ``, ., Tho...  
1    [`, :, :, the, term, ``, standard, model, ``, ...  
2    [True, or, false, ,, the, situation, as, of

### 2.11) Stemming using PorterStemmer

In [21]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
def stem_word(tweet):
    return " ".join([ps.stem(word) for word in tweet.split()])
df['tokens'] = df['tweet'].apply(tokenize_tweet)
print(df)

                                                 tweet  Label  \
0    creative dictionary definitions terms insuranc...      0   
1    term standard model less npov think wed prefer...      0   
2    true false situation march 2002 saudi proposal...      0   
3    next maybe could work less condescending sugge...      0   
4                             page need disambiguation      0   
..                                                 ...    ...   
415  great president ass nigger faggot used white h...      1   
416  trying help people im literally dumbass cunt c...      1   
417                      user ching chong sheng shiong      1   
418                                    yu white nigger      1   
419  glad despicable trump fan magabomber quickly f...      1   

                                                tokens  
0    [creative, dictionary, definitions, terms, ins...  
1    [term, standard, model, less, npov, think, wed...  
2    [true, false, situation, march, 2002, saudi

### 2.12) Lemmatization using WordNetLemmatizer

In [22]:
from nltk.stem import WordNetLemmatizer
WordNetLemmatizer = WordNetLemmatizer()
def stem_word(tweet):
    return " ".join([WordNetLemmatizer.lemmatize(word) for word in tweet.split()])
df['tokens'] = df['tweet'].apply(tokenize_tweet)
print(df)

                                                 tweet  Label  \
0    creative dictionary definitions terms insuranc...      0   
1    term standard model less npov think wed prefer...      0   
2    true false situation march 2002 saudi proposal...      0   
3    next maybe could work less condescending sugge...      0   
4                             page need disambiguation      0   
..                                                 ...    ...   
415  great president ass nigger faggot used white h...      1   
416  trying help people im literally dumbass cunt c...      1   
417                      user ching chong sheng shiong      1   
418                                    yu white nigger      1   
419  glad despicable trump fan magabomber quickly f...      1   

                                                tokens  
0    [creative, dictionary, definitions, terms, ins...  
1    [term, standard, model, less, npov, think, wed...  
2    [true, false, situation, march, 2002, saudi

### 2.13) POS Tagging using SpaCy

In [23]:
import spacy
nlp = spacy.load('en_core_web_sm')
def apply_pos_tagging(text):
    doc = nlp(text)
    pos_tags = [(token.text, token.pos_) for token in doc]
    return pos_tags
df['pos_tags'] = df['tweet'].apply(apply_pos_tagging)
print(df)

                                                 tweet  Label  \
0    creative dictionary definitions terms insuranc...      0   
1    term standard model less npov think wed prefer...      0   
2    true false situation march 2002 saudi proposal...      0   
3    next maybe could work less condescending sugge...      0   
4                             page need disambiguation      0   
..                                                 ...    ...   
415  great president ass nigger faggot used white h...      1   
416  trying help people im literally dumbass cunt c...      1   
417                      user ching chong sheng shiong      1   
418                                    yu white nigger      1   
419  glad despicable trump fan magabomber quickly f...      1   

                                                tokens  \
0    [creative, dictionary, definitions, terms, ins...   
1    [term, standard, model, less, npov, think, wed...   
2    [true, false, situation, march, 2002, sa

# 3) Feature Extraction

### 3.1) CountVectorizer for Feature Extraction Bag of Words

In [24]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(
    max_features=5000,
    min_df=5,
    max_df=0.8,
    ngram_range=(1, 1),
    stop_words='english'
)
cv_features = cv.fit_transform(df['tweet'])
print(cv_features)

  (0, 59)	2
  (0, 274)	1
  (0, 137)	2
  (0, 57)	1
  (0, 234)	1
  (0, 233)	1
  (0, 71)	1
  (0, 235)	1
  (0, 97)	1
  (0, 14)	1
  (0, 252)	2
  (0, 94)	1
  (0, 228)	1
  (0, 63)	1
  (0, 39)	2
  (0, 92)	4
  (0, 122)	4
  (0, 52)	1
  (0, 1)	1
  (0, 136)	1
  (0, 113)	1
  (0, 42)	1
  (0, 263)	1
  (0, 111)	1
  (0, 58)	1
  :	:
  (412, 247)	1
  (413, 97)	1
  (413, 77)	1
  (413, 196)	1
  (414, 255)	1
  (415, 253)	1
  (415, 84)	1
  (415, 154)	1
  (415, 267)	1
  (415, 69)	1
  (415, 18)	1
  (416, 97)	1
  (416, 167)	1
  (416, 248)	1
  (416, 208)	1
  (416, 89)	2
  (416, 50)	1
  (417, 255)	1
  (417, 35)	1
  (417, 36)	1
  (418, 154)	1
  (418, 267)	1
  (419, 232)	1
  (419, 250)	1
  (419, 247)	1


### 3.2) CountVectorizer for Feature Extraction (Bigram)

In [25]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(
    max_features=5000,
    min_df=5,
    max_df=0.8,
    ngram_range=(2, 2),
    stop_words='english'
)
cv_features = cv.fit_transform(df['tweet'])
print(cv_features)

  (0, 2)	1
  (3, 6)	1
  (9, 2)	1
  (27, 6)	1
  (28, 6)	1
  (32, 2)	1
  (33, 2)	1
  (40, 8)	1
  (47, 0)	1
  (52, 0)	1
  (53, 0)	1
  (58, 2)	1
  (61, 0)	1
  (64, 0)	1
  (65, 0)	1
  (68, 0)	1
  (69, 0)	1
  (72, 0)	1
  (73, 0)	1
  (74, 0)	1
  (75, 0)	1
  (76, 0)	1
  (80, 5)	1
  (83, 0)	1
  (86, 0)	1
  :	:
  (341, 5)	1
  (342, 9)	1
  (348, 9)	1
  (351, 9)	1
  (351, 1)	1
  (353, 9)	1
  (355, 9)	2
  (359, 7)	1
  (367, 9)	2
  (369, 9)	1
  (372, 9)	4
  (376, 3)	1
  (378, 5)	1
  (378, 1)	1
  (383, 9)	2
  (384, 9)	1
  (392, 9)	1
  (394, 9)	1
  (398, 2)	1
  (400, 9)	1
  (400, 3)	1
  (402, 9)	1
  (403, 9)	3
  (412, 7)	1
  (417, 1)	1


### 3.3) TF-IDF Vectorizer

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(
    max_features=5000,
    min_df=5,
    max_df=0.8,
    ngram_range=(1, 2),
    stop_words='english'
)
tfidf_features = tfidf.fit_transform(df['tweet'])
print(tfidf_features)

  (0, 61)	0.1728861537594601
  (0, 284)	0.13832422875561723
  (0, 143)	0.2685268470973922
  (0, 59)	0.13832422875561723
  (0, 243)	0.13074579407690437
  (0, 242)	0.11088911577129845
  (0, 74)	0.1342634235486961
  (0, 244)	0.13074579407690437
  (0, 101)	0.09506287599508885
  (0, 15)	0.08524545747528259
  (0, 261)	0.20145914878603163
  (0, 97)	0.10938338654064751
  (0, 236)	0.1160037851018548
  (0, 66)	0.1342634235486961
  (0, 41)	0.2685268470973922
  (0, 95)	0.47182407227936257
  (0, 127)	0.3320085507198559
  (0, 54)	0.1342634235486961
  (0, 1)	0.13074579407690437
  (0, 142)	0.12764302498748878
  (0, 118)	0.12764302498748878
  (0, 44)	0.13832422875561723
  (0, 273)	0.1342634235486961
  (0, 116)	0.09262947732445717
  (0, 60)	0.12764302498748878
  :	:
  (413, 101)	0.5333593934929618
  (413, 80)	0.5981336628935124
  (413, 203)	0.5981336628935124
  (414, 264)	1.0
  (415, 262)	0.36695827205604087
  (415, 87)	0.3973220842133241
  (415, 161)	0.40490734499049763
  (415, 277)	0.45774754009521895

### 3.4) Word2Vec Embeddings (Word2Vec)

In [8]:
import gensim
from gensim.models import Word2Vec
import numpy as np

def tokenize_tweet(tweet):
    return word_tokenize(tweet)
tokenized_tweets = [tokenize_tweet(tweet) for tweet in df['tweet']]
word2vec_model = Word2Vec( sentences=tokenized_tweets, vector_size=100, window=5, min_count=1, workers=4, sg=1, hs=0, negative=5, epochs=30)
print(word2vec_model)
def extract_w2v_features(tweet):
    tweet_vectors = [word2vec_model.wv[word] for word in tweet if word in word2vec_model.wv]
    if not tweet_vectors:
        return np.zeros(word2vec_model.vector_size)
    tweet_vector = np.mean(tweet_vectors, axis=0)
    return tweet_vector

w2v_features = [extract_w2v_features(tweet) for tweet in tokenized_tweets]
w2v_features = np.array(w2v_features)
print(w2v_features.shape)
print(w2v_features[:5])

Word2Vec<vocab=4997, vector_size=100, alpha=0.025>
(420, 100)
[[ 6.67080879e-02  9.46860686e-02  4.83923033e-02  6.00708090e-02
  -1.96887106e-02 -1.98439166e-01  9.63523239e-02  3.56047899e-01
  -2.05058604e-01 -2.07049146e-01 -2.73439325e-02 -2.19710350e-01
  -6.55084103e-03  1.15067707e-02  9.92184654e-02 -5.09678014e-02
   3.56151909e-01 -1.64739802e-01 -1.79918826e-01 -4.33011204e-01
   5.48086613e-02 -4.54768278e-02  3.50753605e-01 -2.47099087e-01
   3.51511650e-02  1.61021983e-03 -2.77756572e-01  2.55365580e-01
  -1.85798720e-01  5.89251369e-02  7.94249400e-02  7.34340958e-03
   2.07520843e-01 -3.21074724e-01 -6.49249777e-02 -5.70996068e-02
   1.50700763e-01 -9.39313415e-03 -6.52604774e-02  1.72344092e-02
   1.27969354e-01 -1.30447254e-01 -1.26004964e-01  5.65673672e-02
   2.44587019e-01 -1.52682766e-01 -2.19352543e-01 -6.22715130e-02
   1.20453596e-01  2.43405089e-01 -3.52280885e-02 -1.33260041e-02
   8.10433403e-02 -2.23423824e-01 -9.35083255e-02 -2.84087192e-03
   2.03825489e

### 3.5) Tokenization using BERT Tokenizer

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True , force_download=True)
max_len = 128
def tokenize_data(texts):
    inputs = tokenizer.batch_encode_plus(texts, padding=True, truncation=True,max_length = max_len, return_tensors='pt')
    return inputs

X_encoded = tokenize_data(df['tweet'].tolist())
print(X_encoded)

# 4) Model Creation

### 4.1) Building and Training BILSTM Model

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense
X = w2v_features
y = df['Label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = Sequential()
model.add(Bidirectional(LSTM(units=64, return_sequences=True), input_shape=(X_train.shape[1], 1)))
model.add(Bidirectional(LSTM(units=32)))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss = 'binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=3, batch_size=32, validation_data=(X_test, y_test))



C:\Users\salman\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/3
11/11 ━━━━━━━━━━━━━━━━━━━━ 26s 335ms/step - accuracy: 0.4903 - loss: 0.6941 - val_accuracy: 0.5476 - val_loss: 0.6897
Epoch 2/3
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 146ms/step - accuracy: 0.5675 - loss: 0.6895 - val_accuracy: 0.7143 - val_loss: 0.6880
Epoch 3/3
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 151ms/step - accuracy: 0.7205 - loss: 0.6788 - val_accuracy: 0.6429 - val_loss: 0.6469


### 4.2) Evaluating the Model

In [31]:
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_test)
y_pred_class = (y_pred > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred_class)
print('Test accuracy:', accuracy)

3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 527ms/step
Test accuracy: 0.6428571428571429


### 4.3) Building and Training BERT Model

In [10]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification, create_optimizer
from sklearn.model_selection import train_test_split

df_train, df_eval = train_test_split(df, test_size=0.2, stratify=df['Label'], random_state=42)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def encode_data(df, tokenizer, max_len=128):
    input_ids = []
    attention_masks = []
    labels = []
    
    for _, row in df.iterrows():
        text = row[0]
        label = row[1]

        encoding = tokenizer.encode_plus(
            text,
            max_length=max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='tf'
        )

        input_ids.append(encoding['input_ids'][0])
        attention_masks.append(encoding['attention_mask'][0])
        labels.append(label)

    input_ids = tf.convert_to_tensor(input_ids)
    attention_masks = tf.convert_to_tensor(attention_masks)
    labels = tf.convert_to_tensor(labels)

    return input_ids, attention_masks, labels

train_input_ids, train_attention_masks, train_labels = encode_data(df_train, tokenizer)
eval_input_ids, eval_attention_masks, eval_labels = encode_data(df_eval, tokenizer)

train_dataset = tf.data.Dataset.from_tensor_slices((
    {
        'input_ids': train_input_ids,
        'attention_mask': train_attention_masks
    },
    train_labels
)).batch(16).shuffle(100).prefetch(tf.data.AUTOTUNE)

eval_dataset = tf.data.Dataset.from_tensor_slices((
    {
        'input_ids': eval_input_ids,
        'attention_mask': eval_attention_masks
    },
    eval_labels
)).batch(16).prefetch(tf.data.AUTOTUNE)

model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

num_train_steps = len(train_dataset) * 3  
optimizer, _ = create_optimizer(
    init_lr=2e-5,
    num_train_steps=num_train_steps,
    num_warmup_steps=0
)


model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
model.fit(train_dataset, validation_data=eval_dataset, epochs=3)


C:\Users\salman\AppData\Roaming\Python\Python312\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
C:\Users\salman\AppData\Local\Temp\ipykernel_1004\3600398965.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = row[0]
C:\Users\salman\AppData\Local\Temp\ipykernel_1004\3600398965.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by p

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3


21/21 [==============================] - 704s 25s/step - loss: 0.4846 - accuracy: 0.8720 - val_loss: 0.3522 - val_accuracy: 0.9048
Epoch 2/3
21/21 [==============================] - 614s 30s/step - loss: 0.2065 - accuracy: 0.9881 - val_loss: 0.1961 - val_accuracy: 0.9762
Epoch 3/3
21/21 [==============================] - 931s 44s/step - loss: 0.1288 - accuracy: 0.9940 - val_loss: 0.1710 - val_accuracy: 0.9643


In [11]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109483778 (417.65 MB)
Trainable params: 109483778 (417.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:

for i, layer in enumerate(model.layers):
    print(f"Layer {i + 1}: {layer.name}")


Layer 1: bert
Layer 2: dropout_37
Layer 3: classifier


In [16]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification, create_optimizer
from sklearn.model_selection import train_test_split

df_train, df_eval = train_test_split(df, test_size=0.2, stratify=df['Label'], random_state=42)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def encode_data(df, tokenizer, max_len=128):
    input_ids = []
    attention_masks = []
    labels = []
    
    for _, row in df.iterrows():
        text = row[0]
        label = row[1]

        encoding = tokenizer.encode_plus(
            text,
            max_length=max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='tf'
        )

        input_ids.append(encoding['input_ids'][0])
        attention_masks.append(encoding['attention_mask'][0])
        labels.append(label)

    input_ids = tf.convert_to_tensor(input_ids)
    attention_masks = tf.convert_to_tensor(attention_masks)
    labels = tf.convert_to_tensor(labels)

    return input_ids, attention_masks, labels

train_input_ids, train_attention_masks, train_labels = encode_data(df_train, tokenizer)
eval_input_ids, eval_attention_masks, eval_labels = encode_data(df_eval, tokenizer)

train_dataset = tf.data.Dataset.from_tensor_slices(({
    'input_ids': train_input_ids,
    'attention_mask': train_attention_masks
}, train_labels)).batch(16).shuffle(100).prefetch(tf.data.AUTOTUNE)

eval_dataset = tf.data.Dataset.from_tensor_slices(({
    'input_ids': eval_input_ids,
    'attention_mask': eval_attention_masks
}, eval_labels)).batch(16).prefetch(tf.data.AUTOTUNE)

model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

for layer in model.layers:
    layer.trainable = False

for layer in model.bert.encoder.layer[-2:]:
    layer.trainable = True

for layer in model.layers:
    print(f"Layer: {layer.name}, Trainable: {layer.trainable}")

num_train_steps = len(train_dataset) * 3  
optimizer, _ = create_optimizer(
    init_lr=2e-5,                  
    num_train_steps=num_train_steps,
    num_warmup_steps=0             
)

model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

model.fit(train_dataset, validation_data=eval_dataset, epochs=3)


C:\Users\salman\AppData\Roaming\Python\Python312\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
C:\Users\salman\AppData\Local\Temp\ipykernel_1004\1442344858.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = row[0]
C:\Users\salman\AppData\Local\Temp\ipykernel_1004\1442344858.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by p

Layer: bert, Trainable: False
Layer: dropout_113, Trainable: False
Layer: classifier, Trainable: False
Epoch 1/3
21/21 [==============================] - 208s 9s/step - loss: 0.6957 - accuracy: 0.5208 - val_loss: 0.6894 - val_accuracy: 0.5595
Epoch 2/3
21/21 [==============================] - 178s 9s/step - loss: 0.6979 - accuracy: 0.5298 - val_loss: 0.6894 - val_accuracy: 0.5595
Epoch 3/3
21/21 [==============================] - 183s 9s/step - loss: 0.6999 - accuracy: 0.5179 - val_loss: 0.6894 - val_accuracy: 0.5595


### 4.4) Saving and Compressing the Model

In [35]:
model.save('saved_model_directory', save_format='tf')

INFO:tensorflow:Assets written to: saved_model_directory\assets


INFO:tensorflow:Assets written to: saved_model_directory\assets


In [36]:
model.save_weights('bert_weights.h5')

In [37]:
import shutil
shutil.make_archive('saved_model', 'zip', 'saved_model_directory')

'c:\\Users\\salman\\Desktop\\Final Year Project\\saved_model.zip'

### 4.5) Prediction Function

In [59]:
def predict(sentence):
    encoding = tokenizer.encode_plus(
        sentence,
        max_length=128,
        padding='max_length',
        truncation=True,
        return_tensors='tf'
    )
    input_ids = encoding['input_ids']
    attention_mask = encoding['attention_mask']

    logits = model(input_ids, attention_mask=attention_mask)[0]
    prediction = tf.argmax(logits, axis=-1).numpy()[0]

    return "Positive" if prediction == 0 else "Negative"

print(predict("ah i am sorry then who is the."))

Positive


In [61]:
import torch
from transformers import DataCollatorWithPadding
from transformers import BertForSequenceClassification, BertTokenizer
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

class HateSpeechDataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer, max_len=128):
        self.df = df
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = self.df.iloc[idx, 0]
        label = self.df.iloc[idx, 1]

        encoding = self.tokenizer.encode_plus(
            text,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

df_train, df_eval = train_test_split(df, test_size=0.2, random_state=42, stratify=df['Label'])
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

train_dataset = HateSpeechDataset(df_train, tokenizer)
eval_dataset = HateSpeechDataset(df_eval, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
eval_loader = DataLoader(eval_dataset, batch_size=16, shuffle=False)

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
loss_fn = torch.nn.CrossEntropyLoss()

epochs = 3
for epoch in range(epochs):
    model.train()
    total_train_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_train_loss += loss.item()

        loss.backward()
        optimizer.step()

    avg_train_loss = total_train_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{epochs} | Avg Train Loss: {avg_train_loss:.4f}")

    model.eval()
    total_eval_loss = 0
    total_correct = 0
    with torch.no_grad():
        for batch in eval_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_eval_loss += loss.item()

            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)
            total_correct += (predictions == labels).sum().item()

    avg_eval_loss = total_eval_loss / len(eval_loader)
    accuracy = total_correct / len(df_eval)
    print(f"Avg Eval Loss: {avg_eval_loss:.4f} | Accuracy: {accuracy:.4f}")


C:\Users\salman\AppData\Roaming\Python\Python312\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3 | Avg Train Loss: 0.5137
Avg Eval Loss: 0.3302 | Accuracy: 0.8571
Epoch 2/3 | Avg Train Loss: 0.1475
Avg Eval Loss: 0.2399 | Accuracy: 0.9167
Epoch 3/3 | Avg Train Loss: 0.0467
Avg Eval Loss: 0.3506 | Accuracy: 0.8810
